# Setup

In [21]:
import pandas as pd
import random

# Load dataset
df = pd.read_csv("03_cleaned_with_images_and_evolutionary_stages.csv")  # your CSV


In [5]:
ATTRIBUTES = ['Generation', 'Height', 'Weight', 'Type1', 'Type2', 'Color', 'evolutionary_stage']


# Solver Core functionality

In [6]:
def get_feedback(secret, guess):
    feedback = {}
    for attr in ATTRIBUTES:
        if pd.isna(secret[attr]) or pd.isna(guess[attr]):
            feedback[attr] = 'gray'
        elif secret[attr] == guess[attr]:
            feedback[attr] = 'green'
        else:
            feedback[attr] = 'gray'
    return feedback


In [ ]:
class PokedleCSP:
    def __init__(self, dataframe, attributes=ATTRIBUTES):
        self.df = dataframe.copy()
        self.attributes = attributes
        self.constraints = {col: [] for col in self.attributes}

    def apply_feedback(self, guess, feedback):
        """Update constraints based on the feedback."""
        for attr, status in feedback.items():
            value = guess[attr]
            if status == "green":
                self.constraints[attr].append(("==", value))
            elif status == "gray":
                self.constraints[attr].append(("!=", value))
            elif status == "yellow":
                # if we later support partial match (like correct type but wrong position)
                self.constraints[attr].append(("in", value))

    def filter_candidates(self):
        """Filter dataset according to constraints."""
        candidates = self.df
        for attr, conds in self.constraints.items():
            for op, val in conds:
                if op == "==":
                    candidates = candidates[candidates[attr] == val]
                elif op == "!=":
                    candidates = candidates[candidates[attr] != val]
                elif op == "in":
                    candidates = candidates[candidates[attr].apply(lambda x: val in str(x))]
        return candidates

    def next_guess(self, candidates):
        """Heuristic: choose Pokémon with most common attributes among remaining candidates."""
        if len(candidates) == 0:
            return None
        return candidates.sample(1).iloc[0]


# Runner

In [8]:
l = []

In [ ]:
def solve(l, attributes=ATTRIBUTES, max_attempts=10):
    solver = PokedleCSP(df, attributes)
    secret = df.sample(1).iloc[0]
    print(f"(DEBUG) Secret Pokémon: {secret['Original_Name']}")

    candidates = df

    for attempt in range(1, max_attempts + 1):
        guess = solver.next_guess(candidates)
        if (len(candidates) == 1):
            print(f"🎯 Guessed correctly! The Pokémon was {guess['Original_Name']}.")
            l.append(attempt)
            break
        if guess is None:
            print(f"💀 No candidates left. Secret was: {secret}")
            l.append(attempt)
            break
        
        feedback = get_feedback(secret, guess)
        print(f"\nAttempt {attempt}: {guess['Original_Name']}")
        print("Feedback:", feedback)
        if all(v == 'green' for v in feedback.values()):
            print(f"🎯 Guessed correctly! The Pokémon was {guess['Original_Name']}.")
            l.append(attempt)
            break
        
        solver.apply_feedback(guess, feedback)
        candidates = solver.filter_candidates()
        print(f"Remaining candidates: {len(candidates)}")


In [ ]:
benchmarks = {}
for i in range(1000):
    solve()

(DEBUG) Secret Pokémon: Carbink

Attempt 1: Aurorus
Feedback: {'Generation': 'green', 'Height': 'gray', 'Weight': 'gray', 'Type1': 'green', 'Type2': 'gray', 'Color': 'gray', 'evolutionary_stage': 'gray'}
Remaining candidates: 4

Attempt 2: Diancie
Feedback: {'Generation': 'green', 'Height': 'gray', 'Weight': 'gray', 'Type1': 'green', 'Type2': 'green', 'Color': 'gray', 'evolutionary_stage': 'green'}
Remaining candidates: 1
🎯 Guessed correctly! The Pokémon was Carbink.
(DEBUG) Secret Pokémon: Poipole

Attempt 1: Stonjourner
Feedback: {'Generation': 'gray', 'Height': 'gray', 'Weight': 'gray', 'Type1': 'gray', 'Type2': 'gray', 'Color': 'gray', 'evolutionary_stage': 'green'}
Remaining candidates: 399

Attempt 2: Girafarig
Feedback: {'Generation': 'gray', 'Height': 'gray', 'Weight': 'gray', 'Type1': 'gray', 'Type2': 'gray', 'Color': 'gray', 'evolutionary_stage': 'green'}
Remaining candidates: 249

Attempt 3: Regice
Feedback: {'Generation': 'gray', 'Height': 'gray', 'Weight': 'gray', 'Type1':

# Benchmarks
- Took 0.0408 seconds per run for CSP with heuristic choose Pokémon with most common attributes among remaining candidates.
- about 6.581892917369308 attempts on an average 

In [18]:
a = sum(l)/len(l)

In [19]:
a

5.889

In [20]:
len(ATTRIBUTES)

7